In [1]:
import pandas as pd

import rpy2
import rpy2.robjects.packages as rpackages
from rpy2.robjects.packages import importr, data
import rpy2.robjects as robjects
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import pandas2ri

In [7]:
utils = importr('utils')
base = importr("base")
datasets = importr('datasets')
ggseg = importr("ggseg")
ggplot2 = importr("ggplot2")
dplyr = importr("dplyr")
tidyr = importr("tidyr")

ggseg3d = importr('ggseg3d')

htmlwidgets = importr('htmlwidgets')

In [3]:
# Installing R packages

utils.chooseCRANmirror(ind=1)
# utils.install_packages('ggseg3d', repos="https://cloud.r-project.org")

<rpy2.rinterface_lib.sexp.NULLType object at 0x7fa5f9e2cb40> [RTYPES.NILSXP]

### Convert PandasDataFrame to rpy2 DataFrame

In [4]:
dk_data = pd.read_csv("data/dk_R_subtype0.csv") 
# dk_data

aseg_data = pd.read_csv("data/aseg_R_subtype0.csv") 
# aseg_data

In [5]:
# Convers DK-atlas
with localconverter(robjects.default_converter + pandas2ri.converter):
    dk_data_R = robjects.conversion.py2rpy(dk_data)
    
# Convers ASEG-atlas
with localconverter(robjects.default_converter + pandas2ri.converter):
    aseg_data_R = robjects.conversion.py2rpy(aseg_data)
    
print(f'Type of dk_data_R: {type(dk_data_R)}')
print(f'Type of aseg_data_R: {type(aseg_data_R)}')

Type of dk_data_R: <class 'rpy2.robjects.vectors.DataFrame'>
Type of aseg_data_R: <class 'rpy2.robjects.vectors.DataFrame'>


## Ggseg 3D - Introduction

#### ggseg3d: Tri-Surface Mesh Plots for Brain Atlases

https://cran.r-project.org/web/packages/ggseg3d/index.html#:~:text=ggseg3d%3A%20Tri%2DSurface%20Mesh%20Plots%20for%20Brain%20Atlases&text=The%20functions%20and%20data%20enable,are%20wrappers%20for%20'plotly'.

Mainly contains a plotting function ggseg3d(), and data of two standard brain atlases (Desikan-Killiany and aseg). By far, the largest bit of the package is the data for each of the atlases. The functions and data enable users to plot tri-surface mesh plots of brain atlases, and customise these by projecting colours onto the brain segments based on values in their own data sets. Functions are wrappers for 'plotly'. Mowinckel & Vidal-Piñeiro (2020) <doi:10.1177/2515245920928009>.

### Function definitions

Definitions of R functions imported to the notebook

In [59]:
print(ggseg3d.ggseg3d)

function (.data = NULL, atlas = "dk_3d", surface = "LCBC", hemisphere = c("right", 
    "subcort"), label = "region", text = NULL, colour = "colour", 
    palette = NULL, na.colour = "darkgrey", na.alpha = 1, show.legend = TRUE, 
    options.legend = NULL, ...) 
{
    atlas3d = get_atlas(atlas, surface = surface, hemisphere = hemisphere)
    if (!is.null(.data)) {
        atlas3d <- data_merge(.data, atlas3d)
    }
    pal.colours <- get_palette(palette)
    if (is.numeric(unlist(atlas3d[, colour]))) {
        if (is.null(names(palette))) {
            pal.colours$values <- seq(min(atlas3d[, colour], 
                na.rm = TRUE), max(atlas3d[, colour], na.rm = TRUE), 
                length.out = nrow(pal.colours))
        }
        atlas3d$new_col = gradient_n_pal(pal.colours$orig, pal.colours$values, 
            "Lab")(unlist(atlas3d[, colour]))
        fill = "new_col"
    }
    else {
        fill = colour
    }
    p = plotly::plot_ly()
    for (tt in 1:nrow(atlas3d)) {
       

In [14]:
# print(ggseg3d.pan_camera)

In [17]:
# print(ggseg3d.get_palette)

In [18]:
# print(ggseg3d.add_glassbrain)

### Get atlases

Import DK-atlas and ASEG-atlas into the notebook

In [8]:
dk_3d = ggseg3d.get_atlas('dk_3d', surface = ["LCBC",'inflated'], hemisphere = ['left','right'])
aseg_3d = ggseg3d.get_atlas('aseg_3d', surface = "LCBC", hemisphere = 'subcort')

# print(f'DK-atlas: {dk_3d}')
# print(f'ASEG-atlas: {aseg_3d}')

### Colors

Define boundry colours for the sequential color pallete (for coloring the regions)

In [9]:
colors = robjects.r('''
c('#6e0101','#ffabab')
''')

print(colors)

[1] "#6e0101" "#ffabab"



In [10]:
# vminmax = [0,25]

colorbar = robjects.r('''
c(0,25)
''')

print(colorbar)

[1]  0 25



### Test ggseg3d function

In [16]:
plot_dk = ggseg3d.ggseg3d(dk_data_R, atlas = dk_3d, surface = ["LCBC",'inflated'], hemisphere = ['left','right'],
               label = 'region', colour = 'p', palette = colors, vminmax = colorbar)

htmlwidgets.saveWidget(plot_dk, "html_outputs/plot_dk.html", selfcontained = False)


# type(plot_dk)

<rpy2.rinterface_lib.sexp.NULLType object at 0x7fa5f9e2cb40> [RTYPES.NILSXP]

In [60]:
plot_aseg = ggseg3d.ggseg3d(aseg_data_R, atlas = aseg_3d, surface = "LCBC", hemisphere = 'subcort',
               label = 'region', colour = 'p', palette = colors, vminmax = [0,25])

print(plot_aseg)

In [209]:
type(plot_dk)

rpy2.robjects.vectors.ListVector

## DK-ATLAS --> Python

In [186]:
# LEFT HEMISPHERE

dk_left = ggseg3d.ggseg3d(dk_data_R, atlas = dk_3d, surface = ["LCBC",'inflated'], hemisphere = 'left',
               label = 'region', colour = 'p', palette = colors)

dk_left = ggseg3d.pan_camera(dk_left, 'right medial')


print(dk_left)

In [190]:
# RIGHT HEMISPHERE

dk_right = ggseg3d.ggseg3d(dk_data_R, atlas = dk_3d, surface = ["LCBC",'inflated'], hemisphere = 'right',
               label = 'region', colour = 'p', palette = colors)

dk_right = ggseg3d.pan_camera(dk_right, 'left medial')


print(dk_right)

In [188]:
# WHOLE BRAIN

dk_full = ggseg3d.ggseg3d(dk_data_R, atlas = dk_3d, surface = ["LCBC",'inflated'], hemisphere = ['left','right'],
               label = 'region', colour = 'p', palette = colors)

print(dk_full)

## ASEG-ATLAS --> Python

In [191]:
# WITHOUT REFERENCE

plot_aseg = ggseg3d.ggseg3d(aseg_data_R, atlas = aseg_3d, surface = "LCBC", hemisphere = 'subcort',
               label = 'region', colour = 'p', palette = colors)

print(plot_aseg)



In [177]:
# ADD GLASSBRAIN

plot_aseg = ggseg3d.add_glassbrain(plot_aseg, hemisphere = ["left", "right"], colour = "#cecece", opacity = 0.3)

print(plot_aseg)